In [1]:
import asyncio
import os
from openai import AsyncOpenAI
import pandas as pd
from jinja2 import Template
import json

In [2]:
import re
from bs4 import BeautifulSoup
from rapidfuzz import fuzz


def normalize_text(text):
    """Normalize text to make it suitable for matching."""
    text = BeautifulSoup(text, "html.parser").get_text()
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = re.sub(r'(\d+),(\d+)', r'\1\2', text)  # Normalize numbers (e.g., 10,000 -> 10000)
    text = re.sub(r'\s+', ' ', text).strip()  # Normalize spaces
    return text

def remove_noise_tokens(snippet):
    """Remove noise tokens such as dates or similar patterns."""
    snippet = re.sub(r'\d{1,2}[./-]\d{1,2}[./-]\d{2,4}', '', snippet)  # Remove dates
    snippet = re.sub(r'—|-', '', snippet)  # Remove long dashes
    snippet = snippet.strip()
    return snippet

In [3]:
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
queries = [
    # Wearable Devices Verification
    "Does Meta produce wearable devices?",
    "List companies that are involved in wearable technology.",
    "Does Apple manufacture wearable gadgets like smartwatches or fitness bands?",
    "Is Samsung a leader in the wearable device market?",
    "Identify companies producing wearable healthcare devices.",

    # Company Verification Queries
    "Give me companies with revenue greater than 100 million USD.",
    "List all companies with annual revenue exceeding $1 billion.",
    "Verify whether Tesla's revenue is greater than $500 million.",
    "Do startups with revenue over $10 million exist in the fintech sector?",
    "Check whether Amazon's revenue surpasses $100 billion.",

    # Industry Verification Queries
    "Does IBM belong to the technology industry?",
    "Verify if ExxonMobil operates in the oil and gas industry.",
    "Is Google classified under the advertising and media sector?",
    "Identify whether companies like Pfizer belong to the pharmaceutical industry.",
    "Check if SpaceX is part of the aerospace and defense industry.",

    # Sector or Market Focus
    "Does Microsoft operate in the cloud computing sector?",
    "Which companies are involved in the green energy industry?",
    "Verify if Facebook is categorized under social media platforms.",

    # Competitive Presence
    "Who are the competitors of Nvidia in the GPU market?",
    "List companies that dominate the e-commerce industry."

    # Factor Search Queries
    "Provide company background for the company bykea.",
    "What are the long term sales goals for bykea?",
    "what are some customer care policies for bykea?",
    "what is the industry and market position of bykea?",
    "what are bykea's carbon neutrality goals"
]


In [5]:
SYSTEM_MESSAGE = """You are an AI assistant specialized in precise question answering based on provided context. Your task is to carefully analyze given context information and answer a specific question with extreme accuracy and thoroughness."""
USER_MESSAGE = """
Here is the context information you should use to answer the question:
<context>
{{CONTEXT_CHUNKS}}
</context>

Now, consider the following question:
<question>
{{QUESTION}}
</question>

To answer this question:
1. Carefully read and analyze the context chunks provided.
2. Identify information that is directly relevant to the question.
3. Formulate a clear, concise, and accurate answer based solely on the information found in the context chunks.
4. Do not include any information or assumptions that are not explicitly stated in the context.
5. If the question asks for a list (especially a list of companies), provide full recall and list all of the companies mentioned in the context, you shouldn't miss any companies.

Provide your answer in the following format:
<answer>
[Your answer here]
</answer>

Guidelines for precision and recall:
- Strive for 100% precision: Every piece of information in your answer must be directly supported by the context.
- Aim for high recall: Include all relevant information from the context that pertains to the question.
- Use exact quotes or paraphrase closely when referring to specific information from the context.
- If the context contains conflicting information, state this explicitly in your answer.

If the context does not contain sufficient information to fully answer the question:
- State clearly what aspects of the question cannot be answered based on the given context.
- Provide any partial information that is available and relevant.
- Do not speculate or provide information beyond what is given in the context.

Remember, your goal is to provide the most accurate and complete answer possible based solely on the information provided in the context chunks."""

In [6]:
SYSTEM_V2 = """You are an AI assistant specialized in precise question answering based on provided context. Your task is to carefully analyze given context information and answer a specific question with extreme accuracy and thoroughness."""
USER_V2 = """
Here is the context information you should use to answer the question:

Google Snippet Context:
<google_snippet>
{{SNIPPET_CHUNKS}}
</google_snippet>

Crawled Webpage Content:
<crawled_webpage>
{{CONTEXT_CHUNKS}}
</crawled_webpage>

Now, consider the following question:
<question>
{{QUESTION}}
</question>

To answer this question, please follow these steps:

1. Carefully read and analyze both the Google snippet and the crawled webpage content.
2. Begin your analysis by examining the Google snippet, as it often contains concise but meaningful answers.
3. If the Google snippet doesn't provide sufficient information, thoroughly analyze the crawled webpage content.
4. Identify all information that is directly relevant to the question.
5. Pay special attention to lists of companies or other entities mentioned in the question, ensuring maximum recall.
6. Formulate a clear, concise, and accurate answer based solely on the information found in the provided context.
7. Do not include any information or assumptions that are not explicitly stated in the context.

Guidelines for precision and recall:
- Strive for 100% precision: Every piece of information in your answer must be directly supported by the context.
- Aim for high recall: Include all relevant information from the context that pertains to the question, especially for lists of entities.
- Use exact quotes or paraphrase closely when referring to specific information from the context.
- If the context contains conflicting information, state this explicitly in your answer.

If the context does not contain sufficient information to fully answer the question:
- State clearly what aspects of the question cannot be answered based on the given context.
- Provide any partial information that is available and relevant.
- Do not speculate or provide information beyond what is given in the context.

Provide your final answer in the following format:
<answer>
[Your answer here: Concise and accurate answer based on the analysis, addressing all aspects of the question if possible]
</answer>


Remember, your goal is to provide the most accurate and complete answer possible based solely on the information provided in the context chunks."""

In [7]:
# TODO: Add heuristic for window length based on the queries e.g., if companies in the snippet are listen one by one, then less window, if companies are along with their descriptions then a little bit wider window
SYSTEM_VALIDATOR_AGENT = """You are an AI assistant specialized in analyzing search queries and snippets to determine the most effective way to answer questions. Your task is to decide whether the provided snippets are sufficient to answer a given query or if expanded snippets are needed for better recall."""
USER_VALIDATOR_AGENT = """
Here is the query you need to analyze:
<query>
{{QUERY}}
</query>

Here are the snippets you need to evaluate:
<snippets>
{{SNIPPETS}}
</snippets>

Carefully analyze the query and the snippets. Consider the following:

1. Is the query asking for a list or comprehensive information that might require high recall?
2. Do the snippets contain enough information to fully answer the query?
3. Would expanding the snippets likely provide additional relevant information?

Based on your analysis, make one of two decisions:

1. If the snippets are sufficient to answer the query:
   Output a JSON object with "answer" based on the question and context provided in the snippet.

2. If the snippets are not sufficient or expanding them would significantly improve recall:
   Output a JSON object with the following properties:
   - "answer_mode": Set to "expanded"
   - "snippet_indices": An array of the top 5 snippet indices (starting from 0) that are most likely to contain relevant information when expanded


Ensure your output is a valid JSON object. Do not include any explanation or additional text outside the JSON object.

Examples of correct outputs:

For a query that can be answered with existing snippets:
{"answer": "provide you answer here based on the query and context with precision and high recall"}

For a query requiring expanded snippets:
{"answer_mode": "expanded", "snippet_indices": [1, 4, 0, 2, 3], }

Provide your decision as a JSON object:"""

In [8]:
SYSTEM_VALIDATOR_AGENT = """You are an AI assistant specialized in analyzing search queries and snippets to determine the most effective way to answer questions. Your task is to decide whether the provided snippets are sufficient to answer a given query or if expanded snippets are needed for better recall."""

USER_VALIDATOR_AGENT = """
Here are the snippets you need to evaluate:
<snippets>
{{SNIPPETS}}
</snippets>

Here is the query you need to analyze:
<query>
{{QUERY}}
</query>


Please analyze the query, and snippets carefully, considering the following points:

1. Is the query asking for a list or comprehensive information that might require high recall? 
2. Do the snippets contain enough information to fully answer the query? Quote the most relevant parts of the snippets.
3. Would expanding the snippets likely provide additional relevant information? 
4. Which snippets are most relevant to the query and title? List them by index (starting from 0).


Based on your analysis, make one of two decisions:

1. If the snippets are sufficient to answer the query:
   Output a JSON object with "answer" based on the question and context provided in the snippet.

2. If the snippets are not sufficient or expanding them would significantly improve recall:
   Output a JSON object with the following properties:
   - "answer_mode": Set to "expanded"
   - "snippet_indices": An array of the top 5 snippet indices (starting from 0) that are most likely to contain relevant information when expanded, considering both the query and the title

Ensure your output is a valid JSON object. Do not include any explanation or additional text outside the JSON object.

Examples of correct outputs:

For a query that can be answered with existing snippets:
{"answer": "provide you answer here based on the query and context with precision and high recall, your answers should be appropriate to questions asked."}

For a query requiring expanded snippets:
{"answer_mode": "expanded", "snippet_indices": [1, 4, 0, 2, 3]}

Provide your decision as a JSON object without ```json ```:"""

In [9]:
async def validator_agent_call(query,  snippet_chunks, temperature=0.1, model="gpt-4o", **kwargs):

    
    user_message = Template(USER_VALIDATOR_AGENT).render({"QUERY" : query, "SNIPPETS": snippet_chunks})
    messages = [
            # {"role": "system", "content": NER_MANAGEMENT_LEVEL_TITLE_SYSTEM_PROMPT},
            {"role": "system", "content": SYSTEM_VALIDATOR_AGENT},
            {"role": "user", "content": f"User Query: {user_message}"},
            
    ]


    openai_object = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
    }
    aclient = AsyncOpenAI(api_key=openai_api_key)

    openai_object.update(kwargs)

    response = await aclient.chat.completions.create(**openai_object)
    response = response.__dict__
    response["choices"] = [choice.__dict__ for choice in response["choices"]]
    for choice in response["choices"]:
        choice["message"] = choice["message"].__dict__
    return response

In [103]:
async def chatgpt_response_a(query, chunks, snippet_chunks = None, temperature=0.1, model="gpt-4o", **kwargs):

    """
    Function to run prompts on chatgpt

    Args:
        key (string): openai api key
        messages (list): list of object that has the chat that you want to process with chatgpt. i.e. system prompt, assistant prompt and user prompt
        temperature (float, optional): Temperature of gpt for generations. Defaults to 0.7.
        model (str, optional): The model you want to use. Defaults to "gpt-4o".

    Returns:
        string: chatgpt result
    """
    # user_message = Template(NER_MANAGEMENT_LEVEL_TITLE_USER_PROMPT).render({"QUERY" : query})
    pre_processed_chunks = []
    for chunk in chunks[:5]:
        pre_processed_chunks.append(normalize_text(remove_noise_tokens(chunk)))
    if snippet_chunks:
        pre_processed_snippet_chunks = []
        for chunk in snippet_chunks[:5]:
            pre_processed_snippet_chunks.append(normalize_text(remove_noise_tokens(chunk)))
        user_message_v2 = Template(USER_V2).render({"QUESTION" : query, "CONTEXT_CHUNKS": pre_processed_chunks, 'SNIPPET_CHUNKS': pre_processed_snippet_chunks})
        messages = [
                # {"role": "system", "content": NER_MANAGEMENT_LEVEL_TITLE_SYSTEM_PROMPT},
                {"role": "system", "content": SYSTEM_V2},
                {"role": "user", "content": f"User Query: {user_message_v2}"},
                
        ]
    else:
        user_message = Template(USER_MESSAGE).render({"QUESTION" : query, "CONTEXT_CHUNKS": pre_processed_chunks})
        messages = [
                # {"role": "system", "content": NER_MANAGEMENT_LEVEL_TITLE_SYSTEM_PROMPT},
                {"role": "system", "content": SYSTEM_MESSAGE},
                {"role": "user", "content": f"User Query: {user_message}"},
                
        ]
    print(messages)
    openai_object = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
    }
    aclient = AsyncOpenAI(api_key=openai_api_key)

    openai_object.update(kwargs)

    response = await aclient.chat.completions.create(**openai_object)
    response = response.__dict__
    response["choices"] = [choice.__dict__ for choice in response["choices"]]
    for choice in response["choices"]:
        choice["message"] = choice["message"].__dict__
    return response

In [104]:
import asyncio
from typing import List, Dict
import math

async def process_batch(texts, snippet_mode=False, v2_mode=True) -> List[Dict,]:
    """Process a single batch of texts using the original title_management logic"""
    ans = []
    tasks = []
    for text in texts:
        chunks = []
        snippet_chunks = []
        if snippet_mode:
            print('Snippet Mode')
            for snippet in text['query_result']:
                chunks.append(f"{snippet['title']} \n {snippet['snippet']}")
            query = text["query"]
            tasks.append(chatgpt_response_a(query, chunks, temperature=0.1, model="gpt-4o"))
        elif v2_mode:
            print("V2 Mode")
            snippet_chunks = text['snippets_list']
            chunks = text['chunks']
            query = text["query"]
            tasks.append(chatgpt_response_a(query, chunks, snippet_chunks=snippet_chunks, temperature=0.1, model="gpt-4o"))

        else:
            print("default mode")
            # chunks = text['chunks'][:2]
            chunks = text['chunks']
            query = text["query"]
            tasks.append(chatgpt_response_a(query, chunks, temperature=0.1, model="gpt-4o"))

    results = await asyncio.gather(*tasks)
    for query, res in zip(texts, results):
        response = res
        response = response["choices"][0]["message"]["content"]
        ans.append({
            "query": query["query"],
            "answer": response,

        })
    return ans


async def process_batch_with_agent_call(texts) -> List[Dict,]:
    """Process a single batch of texts using the original title_management logic"""
    ans = []
    tasks = []
    for text in texts:
        print("validator agent call mode")
        snippet_chunks = text['snippets_list']
        query = text["query"]
        tasks.append(validator_agent_call(query, snippet_chunks, temperature=0.1, model="gpt-4o"))

    results = await asyncio.gather(*tasks)
    for query, res in zip(texts, results):
        response = res
        response = response["choices"][0]["message"]["content"]
        ans.append({
            "query": query["query"],
            "answer": response,

        })
    return ans


async def batched_title_management(texts: List[Dict], snippet_mode ,v2_mode,batch_size: int = 10) -> List[Dict]:
    """Process texts in batches of specified size"""
    all_results = []
    num_batches = math.ceil(len(texts) / batch_size)
    
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(texts))
        batch = texts[start_idx:end_idx]
        print(batch)
        
        # try:
        print("Hello")
        batch_results = await process_batch(batch, snippet_mode, v2_mode)
        print(batch_results)
        all_results.extend(batch_results)
        
        # Optional: Add a small delay between batches to avoid rate limiting
        if i < num_batches - 1:
            await asyncio.sleep(1)
                
        # except Exception as e:
        #     print(f"Error processing batch {i+1}/{num_batches}: {str(e)}")
        #     # You might want to handle the error differently depending on your needs
        #     continue
    
    return all_results

async def batched_title_management_v2(texts: List[Dict], batch_size: int = 10) -> List[Dict]:
    """Process texts in batches of specified size"""
    all_results = []
    num_batches = math.ceil(len(texts) / batch_size)
    
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(texts))
        batch = texts[start_idx:end_idx]
        
        # try:
        batch_results = await process_batch_with_agent_call(batch)
        print(batch_results)
        all_results.extend(batch_results)
        
        # Optional: Add a small delay between batches to avoid rate limiting
        if i < num_batches - 1:
            await asyncio.sleep(1)
                
        # except Exception as e:
        #     print(f"Error processing batch {i+1}/{num_batches}: {str(e)}")
        #     # You might want to handle the error differently depending on your needs
        #     continue
    
    return all_results

In [39]:
empirical_analysis_data = []
with open('empirical_analysis_data_v2.json') as f:
    empirical_analysis_data = json.load(f)

In [40]:
# val1 = await validator_agent_call(query=empirical_analysis_data[2]['query'], snippet_chunks=empirical_analysis_data[2]['snippets_list'], model="gpt-4o-mini")
# val2 = await validator_agent_call(query=empirical_analysis_data[6]['query'], snippet_chunks=empirical_analysis_data[6]['snippets_list'], model="gpt-4o-mini")
# minival1 = await validator_agent_call(query=empirical_analysis_data[2]['query'], snippet_chunks=empirical_analysis_data[2]['snippets_list'], model="gpt-4o")
# minival2 = await validator_agent_call(query=empirical_analysis_data[6]['query'], snippet_chunks=empirical_analysis_data[6]['snippets_list'], model="gpt-4o")
# print(val1['choices'][0]['message']['content'])
# print(val2['choices'][0]['message']['content'])
# print(minival1['choices'][0]['message']['content'])
# print(minival2['choices'][0]['message']['content'])

In [74]:
inference_data = []

In [95]:
empirical_analysis_data = []
with open('crawled_cleaned_content_v2.json') as f:
    empirical_analysis_data = json.load(f)

In [76]:
for item in (empirical_analysis_data):
    inference_data.append({
        "query": item['query'],
        "chunks": item['crawled_chunks'],
        "snippets_list": item['corresponding_snippets'],
    })

In [79]:
for item in inference_data:
    assert len(item['chunks']) == len(item['snippets_list'])

In [80]:
result = await batched_title_management_v2(inference_data, batch_size=10)

validator agent call mode
validator agent call mode
validator agent call mode
validator agent call mode
validator agent call mode
validator agent call mode
validator agent call mode
validator agent call mode
validator agent call mode
validator agent call mode
[{'query': 'Does Meta produce wearable devices?', 'answer': '{"answer": "Yes, Meta produces wearable devices, including AR glasses and other wearable technology, as indicated by their unveiling of \'Orion\' AR glasses and other innovations in wearable computing."}'}, {'query': 'List companies that are involved in wearable technology.', 'answer': '{"answer": "Some companies involved in wearable technology include Tapestry, Doodle Labs, Jabra Hearing, SanMar, Princess Polly, FIT:MATCH.ai, Walking Tree, Cala Health, Hinge Health, Petkit, Whoop, Neosensory, Silvertree, COROS, Apollo, Apple, Epson, Fujitsu, Google, Microsoft, Vuzix, ASUS, Fitbit, Garmin, Honeywell, and Samsung."}'}, {'query': 'Does Apple manufacture wearable gadgets li

In [81]:
expanded_queries = []
non_expanded_queries = []
for res in result:
    answer = json.loads(res['answer'])
    if answer.get("answer_mode"):
        expanded_queries.append({
            "query": res['query'],
            "snippet_indices": json.loads(res['answer'])['snippet_indices']
        })
    else:
        non_expanded_queries.append({
            "query": res['query'],
            "answer": json.loads(res['answer'])['answer']
        })

In [82]:
non_expanded_queries

[{'query': 'Does Meta produce wearable devices?',
  'answer': "Yes, Meta produces wearable devices, including AR glasses and other wearable technology, as indicated by their unveiling of 'Orion' AR glasses and other innovations in wearable computing."},
 {'query': 'List companies that are involved in wearable technology.',
  'answer': 'Some companies involved in wearable technology include Tapestry, Doodle Labs, Jabra Hearing, SanMar, Princess Polly, FIT:MATCH.ai, Walking Tree, Cala Health, Hinge Health, Petkit, Whoop, Neosensory, Silvertree, COROS, Apollo, Apple, Epson, Fujitsu, Google, Microsoft, Vuzix, ASUS, Fitbit, Garmin, Honeywell, and Samsung.'},
 {'query': 'Does Apple manufacture wearable gadgets like smartwatches or fitness bands?',
  'answer': "Yes, Apple manufactures wearable gadgets like smartwatches. The Apple Watch is a notable example, as mentioned in the snippets: 'The Apple Watch made smartwatches the wearable of choice.' and 'Most notably, it developed the Apple Watch

In [83]:
expanded_queries

[{'query': 'Is Samsung a leader in the wearable device market?',
  'snippet_indices': []},
 {'query': 'Give me companies with revenue greater than 100 million USD.',
  'snippet_indices': [0, 3, 6, 4, 2]},
 {'query': 'List all companies with annual revenue exceeding $1 billion.',
  'snippet_indices': [0, 1, 2, 4, 9]},
 {'query': 'Do startups with revenue over $10 million exist in the fintech sector?',
  'snippet_indices': []},
 {'query': 'Is Google classified under the advertising and media sector?',
  'snippet_indices': [0, 1, 2, 6, 7]},
 {'query': 'Who are the competitors of Nvidia in the GPU market?',
  'snippet_indices': []}]

In [84]:
empirical_analysis_data

[{'query': 'Does Meta produce wearable devices?',
  'snippets': ['Meta unveils Orion, its dubbed first true AR glassesAR glassesA pair of smartglasses can be considered an augmented reality device if it performs pose tracking. Superimposing information onto a field of view is achieved through an optical head-mounted display (OHMD) or embedded wireless glasses with transparent heads-up display (HUD) or augmented reality (AR) overlay.https://en.wikipedia.org › wiki › SmartglassesSmartglasses - Wikipedia that look into the future of smartphones as hands-free and wearable AI devices. Mark Zuckerberg led the reveal during his keynote speech at Meta Connect 2024, which ran between September 25th and 26th, 2024.',
   '25.09.2024 — Meta CEO Mark Zuckerberg unveiled an ambitious vision for the future of wearable technology. The event showcased significant innovations.',
   "26.09.2024 — The future of wearable computing will sit on your face in gear that looks like goggles or sunglasses, Faceboo

In [94]:
def extract_relevant_data(expanded_queries, empirical_analysis_list):
    # Step 1: Filter empirical_analysis_list based on expanded_queries
    queries = []
    for item in expanded_queries:
        queries.append(item['query'])
    filtered_empirical_analysis = [
        item for item in empirical_analysis_list if item["query"] in queries
    ]
    # for item, item2 in zip(filtered_empirical_analysis, expanded_queries):
    #     filtered_snippet_list = []
    #     filtered_crawled_list = []
    #     for index in item2['snippet_indices']:
    #         filtered_snippet_list.append(item['snippets'][index] if index < len(item['snippets']) else "")
    #         filtered_crawled_list.append(item['crawled_chunks'][index] if index < len(item['crawled_chunks']) else "")


    #     item['snippets_list'] = filtered_snippet_list
    #     item['crawled_chunks'] = filtered_crawled_list

    return filtered_empirical_analysis


In [87]:
empirical_analysis_data

[{'query': 'Does Meta produce wearable devices?',
  'snippets': ['Meta unveils Orion, its dubbed first true AR glassesAR glassesA pair of smartglasses can be considered an augmented reality device if it performs pose tracking. Superimposing information onto a field of view is achieved through an optical head-mounted display (OHMD) or embedded wireless glasses with transparent heads-up display (HUD) or augmented reality (AR) overlay.https://en.wikipedia.org › wiki › SmartglassesSmartglasses - Wikipedia that look into the future of smartphones as hands-free and wearable AI devices. Mark Zuckerberg led the reveal during his keynote speech at Meta Connect 2024, which ran between September 25th and 26th, 2024.',
   '25.09.2024 — Meta CEO Mark Zuckerberg unveiled an ambitious vision for the future of wearable technology. The event showcased significant innovations.',
   "26.09.2024 — The future of wearable computing will sit on your face in gear that looks like goggles or sunglasses, Faceboo

In [96]:
filtered_empirical_analysis = extract_relevant_data(expanded_queries, empirical_analysis_data)

In [98]:
inference_data = []
for item in (filtered_empirical_analysis):
    inference_data.append({
        "query": item['query'],
        "chunks": item['crawled_chunks'],
        "snippets_list": item['snippets']
    })

In [99]:
inference_data

[{'query': 'Is Samsung a leader in the wearable device market?',
  'chunks': [],
  'snippets_list': ['1 day ago ·   International Data Corporation (IDC) has published an analysis on shipments of wrist-worn wearable devices worldwide in Q1, Q2, and Q3 of 2024. According to the report, Samsung shipped 11.5 million fitness bands and smartwatches in the first, second, and third quarters of 2024 combined, which gave it a market share of 8.3%. ',
   ' With a strong presence in the health and fitness wearable market, Fitbit had been a leader in the wearable industry since early 2014, when the company held about 45 percent of the market share.... ',
   '1 day ago ·   The affordable Galaxy Fit 3 also helped Samsung expand its footsteps in the smart wearable segment in key markets. As per the chart, Huawei was the largest smartwatch vendor during Q1 to Q3 with 23.6 million units shipment and 16.9% market share. Apple ranked second with 22.5 million shipments and a 16.2% market share during the s

In [105]:
result = await batched_title_management(inference_data, batch_size=10, snippet_mode=False ,v2_mode=True)
result.extend(non_expanded_queries)

[{'query': 'Is Samsung a leader in the wearable device market?', 'chunks': [], 'snippets_list': ['1 day ago ·   International Data Corporation (IDC) has published an analysis on shipments of wrist-worn wearable devices worldwide in Q1, Q2, and Q3 of 2024. According to the report, Samsung shipped 11.5 million fitness bands and smartwatches in the first, second, and third quarters of 2024 combined, which gave it a market share of 8.3%. ', ' With a strong presence in the health and fitness wearable market, Fitbit had been a leader in the wearable industry since early 2014, when the company held about 45 percent of the market share.... ', '1 day ago ·   The affordable Galaxy Fit 3 also helped Samsung expand its footsteps in the smart wearable segment in key markets. As per the chart, Huawei was the largest smartwatch vendor during Q1 to Q3 with 23.6 million units shipment and 16.9% market share. Apple ranked second with 22.5 million shipments and a 16.2% market share during the same time p

In [64]:
filtered_empirical_analysis

[{'query': 'Is Samsung a leader in the wearable device market?',
  'urls': [],
  'content': [],
  'snippets': [],
  'filtered_content': [],
  'snippets_list': [],
  'crawled_chunks': []},
 {'query': 'Give me companies with revenue greater than 100 million USD.',
  'urls': ['https://en.wikipedia.org/wiki/List_of_largest_companies_by_revenue',
   'https://stockanalysis.com/list/highest-revenue/',
   'https://www.cience.com/companies-database/united-states/retail/revenue-50m-100m'],
  'content': ['From Wikipedia, the free encyclopediaWalmarthas been the world\'s largest company by revenue since 2014.[1]This list comprises the world\'s largest companies byconsolidatedrevenue, according to theFortuneGlobal 5002024 rankings and other sources.[2]American retail corporationWalmarthas been the world\'s largest company by revenue since 2014.[1]The list is limited to the largest 50 companies, all of which have annual revenues exceeding US$130 billion. This list is incomplete, as not all companies

In [106]:
len(result)

20

In [107]:
from IPython.core.display import display, HTML

def display_query_answer_table(data):
    """
    Displays a list of dictionaries with keys {'query', 'answer'} in a Jupyter Notebook as an HTML table.
    
    Args:
        data (list of dict): A list where each dict contains 'query' and 'answer' keys.
    """
    table_html = """
    <style>
        table {
            width: 100%;
            border-collapse: collapse;
            font-family: Arial, sans-serif;
        }
        th, td {
            border: 1px solid #ddd;
            padding: 8px;
            text-align: left;
        }
        th {
            background-color: #0d0d0d;
            color: white;
        }
        tr:nth-child(even) {
            background-color: #020202;
        }
    </style>
    <table>
        <tr>
            <th>Query</th>
            <th>Answer</th>
        </tr>
    """
    
    for item in data:
        table_html += f"<tr><td>{item['query']}</td><td>{item['answer']}</td></tr>"
    
    table_html += "</table>"
    
    display(HTML(table_html))


display_query_answer_table(result)


/var/folders/4h/c8xgg6gj0_jbc09wh4qlg7sw0000gn/T/ipykernel_40318/1390082193.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Query,Answer
Is Samsung a leader in the wearable device market?,"Based on the provided context, Samsung is not the leader in the wearable device market. According to the Google snippet, Huawei was the largest smartwatch vendor during the first three quarters of 2024 with 23.6 million units shipped and a 16.9% market share. Apple ranked second with 22.5 million shipments and a 16.2% market share. Samsung shipped 11.5 million fitness bands and smartwatches during the same period, which gave it an 8.3% market share. Therefore, Samsung is not the leader, as both Huawei and Apple have higher market shares in the wearable device market according to the data provided."
Give me companies with revenue greater than 100 million USD.,"Based on the provided context, the following companies in the United States have revenues greater than 100 million USD: 1. Walmart Inc. 2. Amazon.com Inc. 3. UnitedHealth Group Incorporated 4. Apple Inc. 5. Berkshire Hathaway Inc. 6. CVS Health Corporation 7. Exxon Mobil Corporation 8. Alphabet Inc. 9. McKesson Corporation 10. Cencora Inc. 11. Costco Wholesale Corporation 12. Microsoft Corporation These companies are mentioned in the context as having significant revenues, with some exceeding 100 billion USD. The context also notes that 87% of firms with revenue greater than 100 million USD in the US are private, but specific private companies are not listed in the provided information."
List all companies with annual revenue exceeding $1 billion.,"Based on the provided context, the following companies are mentioned as having annual revenues exceeding $1 billion: 1. Nike (United States, Oregon) 2. Ralph Lauren (United States, New York) The context also refers to lists of companies and startups with more than $1 billion in revenue, such as the top 10,000 companies in the United States by Crunchbase rank and the Fortune Global 500 list. However, specific company names from these lists are not provided in the context. Therefore, the answer is limited to the companies explicitly mentioned: Nike and Ralph Lauren. Additional companies with revenues exceeding $1 billion may exist in the referenced lists, but they are not detailed in the provided context."
Do startups with revenue over $10 million exist in the fintech sector?,"Yes, startups with revenue over $10 million exist in the fintech sector. The Google snippet mentions Falcon, a fintech infrastructure company, which has acquired 40 enterprise customers and is earning $10 million in annual revenue."
Is Google classified under the advertising and media sector?,"Based on the provided context, Google can be classified under the advertising sector. The Google snippet and the crawled webpage content both highlight Google's dominant position in the advertising market, particularly in digital advertising and search ads. The snippet mentions Google's control over a significant portion of the ad tech supply chain and its exploration of generating revenue through AI-powered search engines. Additionally, the crawled webpage content discusses Google's main source of revenue being advertising and its involvement in digital advertising markets. However, the context does not explicitly classify Google under the media sector, although it does mention debates about Google being a media company. Therefore, while Google is clearly part of the advertising sector, its classification under the media sector is not definitively supported by the provided context."
Who are the competitors of Nvidia in the GPU market?,"Based on the provided context, the competitors of Nvidia in the GPU market include Advanced Micro Devices (AMD), which is described as Nvidia's direct competitor in the GPU, data center, and AI hardware markets. Additionally, Cerebras Systems is mentioned as differentiating itself from Nvidia through its unique chip architecture, although it is not explicitly stated as a direct competitor in the GPU market. The context does not provide further information about other comp

In [22]:
print(json.dumps(result, indent=2))

[
  {
    "query": "Is Samsung a leader in the wearable device market?",
    "answer": "<answer>\nBased on the provided context, Samsung is not the leader in the wearable device market. According to the Google snippet, Huawei was the largest smartwatch vendor from Q1 to Q3 of 2024 with 23.6 million units shipped and a 16.9% market share, while Apple ranked second with 22.5 million shipments and a 16.2% market share. Samsung shipped 11.5 million fitness bands and smartwatches during the same period, which gave it an 8.3% market share. Therefore, Samsung is not the leader but is a significant player in the wearable device market.\n</answer>"
  },
  {
    "query": "Give me companies with revenue greater than 100 million USD.",
    "answer": "<answer>\nBased on the provided context, several companies with revenue greater than 100 million USD are mentioned. These include:\n\n1. Walmart Inc. with a revenue of 673.82 billion USD.\n2. Amazon.com Inc. with a revenue of 620.13 billion USD.\n3. U

In [77]:
with open('expanded_snippets_4o_cleaned.json' ,'w') as f:
    json.dump(result, f, indent=2)

In [95]:
with open('bm25_markdown_results.json', 'w') as f:
    json.dump(result, f, indent=2)

In [24]:
bm25_results = []
snippet_results = []
with open('bm25_markdown_results.json') as f:
   bm25_results = json.load(f)
with open('snippets_only_results.json') as f:
   snippet_results = json.load(f)

In [99]:
print(bm25_results[0]["answer"])

<answer>
The context does not explicitly state that Meta produces wearable devices. However, it mentions "Meta true AR glasses" as part of a technology discussion, which implies that Meta is involved in the development of wearable technology. Therefore, while it suggests that Meta may produce wearable devices, it does not provide definitive confirmation.
</answer>


In [237]:
queries = []
bm25_answers = []
snippet_answers = []
expanded_snippets_answers = []

for item in bm25_results:
    queries.append(item["query"])
    bm25_answers.append(item["answer"])
for item in snippet_results:
    snippet_answers.append(item["answer"])

for item in result:
    expanded_snippets_answers.append(item['answer'])

In [238]:
import pandas as pd
from IPython.core.display import display, HTML

def display_columns(queries, results1, results2, indices=None):
    """
    Display selected rows of three lists side by side in columns as an HTML table 
    in Jupyter Notebook and return a DataFrame containing the data.
    
    Parameters:
    - queries: List of query strings
    - results1: List of result strings corresponding to the first set
    - results2: List of result strings corresponding to the second set
    - indices: List of integer indices specifying which rows to display (optional)
    
    Returns:
    - pd.DataFrame: A DataFrame with the three lists as columns, filtered by indices if provided.
    """
    # Verify that all lists have the same length
    if not (len(queries) == len(results1) == len(results2)):
        raise ValueError("All input lists must have the same length.")
    
    # Create a DataFrame
    data = {'Query': queries, 'Snippet Only Answers': results1, 'BM25 Answers': results2}
    df = pd.DataFrame(data)
    
    # Filter the DataFrame if indices are provided
    if indices is not None:
        df = df.iloc[indices]  # Select only specified indices
    
    # Generate the HTML table
    html_content = """
    <style>
        table {
            border-collapse: collapse;
            width: 100%;
            text-align: left;
        }
        th, td {
            border: 1px solid #dddddd;
            padding: 8px;
        }
        th {
            background-color: #020202;
            color: white;
        }
    </style>
    <table>
        <thead>
            <tr>
                <th>Query</th>
                <th>Snippet Only Answers</th>
                <th>Expanded Snippet + Actual Snippet Answers</th>
            </tr>
        </thead>
        <tbody>
    """
    
    for _, row in df.iterrows():
        html_content += f"""
        <tr>
            <td>{row['Query']}</td>
            <td>{row['Snippet Only Answers']}</td>
            <td>{row['BM25 Answers']}</td>
        </tr>
        """
    
    html_content += """
        </tbody>
    </table>
    """
    
    # Display the HTML content
    display(HTML(html_content))
    
    # Return the filtered DataFrame
    return df

# Call the function and capture the returned DataFrame
df = display_columns(queries, snippet_answers, expanded_snippets_answers)

"Is Samsung a leader in the wearable device market?"
"Do startups with revenue over $10 million exist in the fintech sector?"
"Who are the competitors of Nvidia in the GPU market?"

/var/folders/4h/c8xgg6gj0_jbc09wh4qlg7sw0000gn/T/ipykernel_97954/760395339.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Query,Snippet Only Answers,Expanded Snippet + Actual Snippet Answers
Does Meta produce wearable devices?,"Yes, Meta produces wearable devices. The context mentions that Meta unveiled its first true AR glasses, dubbed Orion, during a keynote speech by Mark Zuckerberg at Meta Connect 2024. Additionally, it discusses Meta's focus on wearable technology and the creation of a dedicated ""Wearables"" department, indicating a commitment to developing wearable devices. Furthermore, it highlights that Meta's wearables, such as the Ray-Bans, are performing well in the market.","Yes, Meta does produce wearable devices. According to the provided context, Meta has unveiled its augmented reality glasses called Orion, which are considered a significant innovation in wearable technology. Additionally, Meta has been successful with its Ray-Ban smart glasses, which continue to sell well. The company is also involved in developing neural interface technology, as indicated by its neural wristband prototype for smart glasses. Furthermore, Meta has a dedicated wearables department, separate from its metaverse and gaming teams, highlighting its commitment to wearable technology."
List companies that are involved in wearable technology.,"The companies involved in wearable technology mentioned in the context include: 1. Tapestry 2. Doodle Labs 3. Jabra Hearing 4. SanMar 5. Princess Polly 6. FIT:MATCH.ai 7. Walking Tree 8. Cala Health 9. StrongArm Technologies, Inc. 10. Hinge Health 11. Petkit 12. Whoop 13. Neosensory 14. Silvertree 15. COROS 16. Apollo 17. Apple 18. Epson 19. Fujitsu 20. Google 21. Microsoft 22. Vuzix 23. ASUS 24. Fitbit 25. Garmin 26. Honeywell 27. WHOOP 28. Neurable 29. SAMSUNG These companies are involved in various aspects of wearable technology, including health, fitness, and industrial applications.","Based on the provided context, the companies involved in wearable technology include: 1. Apple 2. Epson 3. Fujitsu 4. Google 5. Microsoft 6. Vuzix 7. Asus 8. Fitbit (now owned by Google) 9. Garmin 10. Honeywell 11. Samsung 12. StrongArm Technologies 13. Hinge Health 14. Petkit 15. Whoop 16. Neosensory 17. Silvertree 18. Coros 19. Apollo These companies are mentioned in the context as being involved in the development or manufacturing of wearable technology. If there are any additional companies or specific details required, the context does not provide further information."
Does Apple manufacture wearable gadgets like smartwatches or fitness bands?,"Yes, Apple manufactures wearable gadgets such as smartwatches, specifically the Apple Watch, which has made smartwatches the wearable of choice. The context also mentions the possibility of Apple releasing a purpose-built fitness tracker, indicating their involvement in fitness bands as well. Additionally, wearable technology includes both smartwatches and fitness trackers, which are part of Apple's product offerings.","Yes, Apple manufactures wearable gadgets like smartwatches. The context explicitly mentions that the Apple Watch is a smartwatch and has become the most popular smartwatch in the world. Additionally, the context refers to Apple's dominance in the wearable technology market, particularly with smartwatches. There is no specific mention of Apple manufacturing fitness bands in the provided context."
Is Samsung a leader in the wearable device market?,"Samsung is not a leader in the wearable device market. According to the context, while Samsung shipped 11.5 million fitness bands and smartwatches in the first three quarters of 2024, it held a market share of only 8.3%. In comparison, Huawei was the largest smartwatch vendor during the same period with 23.6 million units shipped and a 16.9% market share, while Apple ranked second with 22.5 million shipments and a 16.2% market share. Therefore, Samsung ranks behind both Huawei and Apple in the wearable device market.","Based on the provided context, Samsung is not the leader in the wearable device market. According to the Google

'Who are the competitors of Nvidia in the GPU market?'

In [144]:
df.to_csv("first iteration comparison.csv")